In [4]:
# Load the datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the aliens dataset
aliens_df = pd.read_csv('../data/ben10_aliens.csv')
print("Aliens Dataset Shape:", aliens_df.shape)
print("\nAliens Data Sample:")
aliens_df.head()

Aliens Dataset Shape: (74, 7)

Aliens Data Sample:


,alien_id,alien_name,species,home_planet,strength_level,speed_level,intelligence
0,1,Alien X,Indeedite,Andromeda Prime,7,8,6
1,2,AmpFibian,Particularite,Zantron,5,8,5
2,3,Armodrillo,Discussionite,Tierra Nova,5,4,10
3,4,Astrodactyl,Acrossite,Hala,10,7,8
4,5,Atomix,Eastite,Sirius,10,5,10


In [2]:
aliens_df.head()

,alien_id,alien_name,species,home_planet,strength_level,speed_level,intelligence
0,1,Alien X,Indeedite,Andromeda Prime,7,8,6
1,2,AmpFibian,Particularite,Zantron,5,8,5
2,3,Armodrillo,Discussionite,Tierra Nova,5,4,10
3,4,Astrodactyl,Acrossite,Hala,10,7,8
4,5,Atomix,Eastite,Sirius,10,5,10


In [3]:
# Load battles dataset
battles_df = pd.read_csv('../data/ben10_battles.csv')
print("Battles Dataset Shape:", battles_df.shape)
print("\nBattles Data Sample:")
battles_df.head()

Battles Dataset Shape: (300, 5)

Battles Data Sample:


,battle_id,alien_name,enemy_name,battle_date,winner
0,1,Rath,Eatle,01-01-2023,Rath
1,2,Lodestar,Wildmutt,31-10-2022,Lodestar
2,3,Walkatrout,Grey Matter,04-07-2024,Grey Matter
3,4,Four Arms,Wildmutt,07-10-2024,Wildmutt
4,5,Shock Rock,Eatle,10-07-2022,Eatle


In [6]:
# Step 1: Clean and prepare aliens data
# Remove unnecessary columns and create composite power score
aliens_clean = aliens_df.copy()
aliens_clean = aliens_clean.drop(['species', 'home_planet'], axis=1)
aliens_clean['power'] = aliens_clean['strength_level'] + aliens_clean['speed_level'] + aliens_clean['intelligence']

print("Cleaned Aliens Data:")
print(f"\nTotal unique aliens: {len(aliens_clean)}")

# Check for duplicates in alien names
print(f"Duplicate alien names: {aliens_clean['alien_name'].duplicated().sum()}")
print(f"Unique alien names: {aliens_clean['alien_name'].nunique()}")
aliens_clean.head()

Cleaned Aliens Data:

Total unique aliens: 74
Duplicate alien names: 0
Unique alien names: 74


,alien_id,alien_name,strength_level,speed_level,intelligence,power
0,1,Alien X,7,8,6,21
1,2,AmpFibian,5,8,5,18
2,3,Armodrillo,5,4,10,19
3,4,Astrodactyl,10,7,8,25
4,5,Atomix,10,5,10,25


In [7]:
# Step 2: Clean battles data
battles_clean = battles_df.copy()
battles_clean = battles_clean.drop(['battle_date'], axis=1)
battles_clean = battles_clean.rename(columns={'alien_name': 'alien_1', 'enemy_name': 'alien_2'})

print("Cleaned Battles Data:")

# Check for missing values
print(f"\nMissing values in battles:")
print(battles_clean.isnull().sum())

# Check unique aliens in battles vs aliens dataset
battle_aliens = set(battles_clean['alien_1'].unique()) | set(battles_clean['alien_2'].unique())
alien_names = set(aliens_clean['alien_name'].unique())

print(f"\nAliens in battles dataset: {len(battle_aliens)}")
print(f"Aliens in aliens dataset: {len(alien_names)}")
print(f"Aliens in battles but not in aliens dataset: {battle_aliens - alien_names}")
print(f"Aliens in aliens dataset but not in battles: {alien_names - battle_aliens}")
battles_clean.head()

Cleaned Battles Data:

Missing values in battles:
battle_id    0
alien_1      0
alien_2      0
winner       0
dtype: int64

Aliens in battles dataset: 74
Aliens in aliens dataset: 74
Aliens in battles but not in aliens dataset: set()
Aliens in aliens dataset but not in battles: set()


,battle_id,alien_1,alien_2,winner
0,1,Rath,Eatle,Rath
1,2,Lodestar,Wildmutt,Lodestar
2,3,Walkatrout,Grey Matter,Grey Matter
3,4,Four Arms,Wildmutt,Wildmutt
4,5,Shock Rock,Eatle,Eatle


In [8]:
# Step 3: Create feature mapping from aliens to battles
# Create a mapping dictionary for quick lookup
alien_stats_map = {}
for _, row in aliens_clean.iterrows():
    alien_stats_map[row['alien_name']] = {
        'strength_level': row['strength_level'],
        'speed_level': row['speed_level'],
        'intelligence': row['intelligence'],
        'power': row['power']
    }

# Add stats for alien_1
battles_clean['alien_1_strength'] = battles_clean['alien_1'].map(lambda x: alien_stats_map[x]['strength_level'])
battles_clean['alien_1_speed'] = battles_clean['alien_1'].map(lambda x: alien_stats_map[x]['speed_level'])
battles_clean['alien_1_intelligence'] = battles_clean['alien_1'].map(lambda x: alien_stats_map[x]['intelligence'])
battles_clean['alien_1_power'] = battles_clean['alien_1'].map(lambda x: alien_stats_map[x]['power'])

# Add stats for alien_2
battles_clean['alien_2_strength'] = battles_clean['alien_2'].map(lambda x: alien_stats_map[x]['strength_level'])
battles_clean['alien_2_speed'] = battles_clean['alien_2'].map(lambda x: alien_stats_map[x]['speed_level'])
battles_clean['alien_2_intelligence'] = battles_clean['alien_2'].map(lambda x: alien_stats_map[x]['intelligence'])
battles_clean['alien_2_power'] = battles_clean['alien_2'].map(lambda x: alien_stats_map[x]['power'])

print("Battles with alien stats:")
battles_clean.head()

Battles with alien stats:


,battle_id,alien_1,alien_2,winner,alien_1_strength,alien_1_speed,alien_1_intelligence,alien_1_power,alien_2_strength,alien_2_speed,alien_2_intelligence,alien_2_power
0,1,Rath,Eatle,Rath,5,8,8,21,5,7,5,17
1,2,Lodestar,Wildmutt,Lodestar,6,8,10,24,8,6,10,24
2,3,Walkatrout,Grey Matter,Grey Matter,5,9,9,23,8,4,6,18
3,4,Four Arms,Wildmutt,Wildmutt,8,4,5,17,8,6,10,24
4,5,Shock Rock,Eatle,Eatle,7,6,6,19,5,7,5,17


In [9]:
# Step 4: Feature Engineering - Create difference features
battles_clean['power_diff'] = battles_clean['alien_1_power'] - battles_clean['alien_2_power']
battles_clean['strength_diff'] = battles_clean['alien_1_strength'] - battles_clean['alien_2_strength']
battles_clean['speed_diff'] = battles_clean['alien_1_speed'] - battles_clean['alien_2_speed']
battles_clean['intelligence_diff'] = battles_clean['alien_1_intelligence'] - battles_clean['alien_2_intelligence']

# Create binary target variable (1 if alien_1 wins, 0 if alien_2 wins)
battles_clean['winner_binary'] = (battles_clean['winner'] == battles_clean['alien_1']).astype(int)

print("Feature engineered battles data:")
print(battles_clean[['alien_1', 'alien_2', 'winner', 'power_diff', 'strength_diff', 'speed_diff', 'intelligence_diff', 'winner_binary']].head(10))

# Check target distribution
print(f"\nTarget distribution:")
print(battles_clean['winner_binary'].value_counts())
print(f"Percentage of alien_1 wins: {battles_clean['winner_binary'].mean():.2%}")

Feature engineered battles data:
      alien_1           alien_2       winner  power_diff  strength_diff  \
0        Rath             Eatle         Rath           4              0   
1    Lodestar          Wildmutt     Lodestar           0             -2   
2  Walkatrout       Grey Matter  Grey Matter           5             -3   
3   Four Arms          Wildmutt     Wildmutt          -7              0   
4  Shock Rock             Eatle        Eatle           2              2   
5   Terraspin  Ultimate Way Big    Terraspin           6              3   
6     Way Big         Four Arms      Way Big           8             -1   
7    Overflow         Big Chill     Overflow          -1             -3   
8  Gravattack          Wildmutt   Gravattack          -6             -3   
9  Gravattack          Wildmutt   Gravattack          -6             -3   

   speed_diff  intelligence_diff  winner_binary  
0           1                  3              1  
1           2                  0         

In [10]:
# Step 5: Prepare features for machine learning
# Select relevant features for the model
feature_columns = [
    'alien_1_strength', 'alien_1_speed', 'alien_1_intelligence', 'alien_1_power',
    'alien_2_strength', 'alien_2_speed', 'alien_2_intelligence', 'alien_2_power',
    'power_diff', 'strength_diff', 'speed_diff', 'intelligence_diff'
]

X = battles_clean[feature_columns]
y = battles_clean['winner_binary']

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("\nFeature columns:")
for i, col in enumerate(feature_columns):
    print(f"{i+1}. {col}")

# Check for any missing values
print(f"\nMissing values in features:")
print(X.isnull().sum().sum())

print(f"\nFeature statistics:")
print(X.describe())

Features shape: (300, 12)
Target shape: (300,)

Feature columns:
1. alien_1_strength
2. alien_1_speed
3. alien_1_intelligence
4. alien_1_power
5. alien_2_strength
6. alien_2_speed
7. alien_2_intelligence
8. alien_2_power
9. power_diff
10. strength_diff
11. speed_diff
12. intelligence_diff

Missing values in features:
0

Feature statistics:
       alien_1_strength  alien_1_speed  alien_1_intelligence  alien_1_power  \
count        300.000000     300.000000            300.000000     300.000000   
mean           7.810000       6.766667              7.530000      22.106667   
std            1.678279       1.993020              1.680828       3.157296   
min            5.000000       4.000000              5.000000      14.000000   
25%            7.000000       5.000000              6.000000      20.000000   
50%            8.000000       7.000000              8.000000      22.000000   
75%            9.000000       8.000000              9.000000      25.000000   
max           10.000000   

In [11]:
# Step 6: Split data and train models
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("Training target distribution:", y_train.value_counts().values)
print("Test target distribution:", y_test.value_counts().values)

# Train Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # Handle slight class imbalance
)

rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nRandom Forest Accuracy: {accuracy:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Training set size: (240, 12)
Test set size: (60, 12)
Training target distribution: [124 116]
Test target distribution: [31 29]

Random Forest Accuracy: 0.4833

Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.55      0.51        29
           1       0.50      0.42      0.46        31

    accuracy                           0.48        60
   macro avg       0.49      0.49      0.48        60
weighted avg       0.49      0.48      0.48        60



In [12]:
# Step 7: Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance Rankings:")
print(feature_importance)

# Try with different model - XGBoost for comparison
try:
    from xgboost import XGBClassifier
    
    xgb_model = XGBClassifier(
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
    
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    xgb_accuracy = accuracy_score(y_test, xgb_pred)
    
    print(f"\nXGBoost Accuracy: {xgb_accuracy:.4f}")
    
except ImportError:
    print("\nXGBoost not available, skipping comparison")

# Let's also try with just the most important features
top_features = feature_importance.head(6)['feature'].tolist()
print(f"\nTop 6 features: {top_features}")

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

rf_model_simple = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)

rf_model_simple.fit(X_train_top, y_train)
y_pred_simple = rf_model_simple.predict(X_test_top)
accuracy_simple = accuracy_score(y_test, y_pred_simple)

print(f"Random Forest with top features accuracy: {accuracy_simple:.4f}")

Feature Importance Rankings:
                 feature  importance
8             power_diff    0.104274
11     intelligence_diff    0.099277
7          alien_2_power    0.098625
10            speed_diff    0.094740
3          alien_1_power    0.091328
1          alien_1_speed    0.083838
9          strength_diff    0.083022
2   alien_1_intelligence    0.080590
0       alien_1_strength    0.069687
6   alien_2_intelligence    0.068244
5          alien_2_speed    0.067355
4       alien_2_strength    0.059022

XGBoost Accuracy: 0.5000

Top 6 features: ['power_diff', 'intelligence_diff', 'alien_2_power', 'speed_diff', 'alien_1_power', 'alien_1_speed']
Random Forest with top features accuracy: 0.5000


In [13]:
# Step 8: Try different approaches to improve accuracy

# 1. Let's check if there are patterns in the data we're missing
print("Analyzing battle patterns...")

# Check win rates by power differences
battles_clean['power_diff_bucket'] = pd.cut(battles_clean['power_diff'], 
                                           bins=[-float('inf'), -5, -2, 2, 5, float('inf')], 
                                           labels=['Very Weak (-5+)', 'Weak (-2 to -5)', 'Even (-2 to +2)', 'Strong (+2 to +5)', 'Very Strong (+5+)'])

power_win_rates = battles_clean.groupby('power_diff_bucket')['winner_binary'].agg(['count', 'mean']).round(3)
print("\nWin rates by power difference:")
print(power_win_rates)

# 2. Try a different approach - Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Scale features for logistic regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_model = LogisticRegression(random_state=42, class_weight='balanced')
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_pred)

print(f"\nLogistic Regression Accuracy: {lr_accuracy:.4f}")

# 3. Try with simplified features focusing on differences only
diff_features = ['power_diff', 'strength_diff', 'speed_diff', 'intelligence_diff']
X_train_diff = X_train[diff_features]
X_test_diff = X_test[diff_features]

rf_diff = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_diff.fit(X_train_diff, y_train)
diff_pred = rf_diff.predict(X_test_diff)
diff_accuracy = accuracy_score(y_test, diff_pred)

print(f"Random Forest with difference features only: {diff_accuracy:.4f}")

Analyzing battle patterns...

Win rates by power difference:
                   count   mean
power_diff_bucket              
Very Weak (-5+)       72  0.500
Weak (-2 to -5)       28  0.607
Even (-2 to +2)       85  0.494
Strong (+2 to +5)     74  0.514
Very Strong (+5+)     41  0.537

Logistic Regression Accuracy: 0.5667
Random Forest with difference features only: 0.5500


C:\Users\wajid\AppData\Local\Temp\ipykernel_12404\4048649493.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  power_win_rates = battles_clean.groupby('power_diff_bucket')['winner_binary'].agg(['count', 'mean']).round(3)


In [15]:
# Step 9: Save the best model and create prediction function
import joblib

# The Logistic Regression performed best, so let's save it
print("Saving the best model (Logistic Regression)...")

# Save the model and scaler
joblib.dump(lr_model, '../models/ben10_battle_predictor.pkl')
joblib.dump(scaler, '../models/ben10_feature_scaler.pkl')
joblib.dump(aliens_clean, '../models/ben10_aliens_clean.pkl')

print("Models saved successfully!")

# Create a prediction function
def predict_battle_winner(alien1_name, alien2_name, aliens_data, model, scaler):
    """
    Predict the winner of a battle between two aliens
    """
    try:
        # Get alien stats
        alien1_stats = aliens_data[aliens_data['alien_name'] == alien1_name].iloc[0]
        alien2_stats = aliens_data[aliens_data['alien_name'] == alien2_name].iloc[0]
        
        # Create feature vector
        features = np.array([
            alien1_stats['strength_level'], alien1_stats['speed_level'], alien1_stats['intelligence'], alien1_stats['power'],
            alien2_stats['strength_level'], alien2_stats['speed_level'], alien2_stats['intelligence'], alien2_stats['power'],
            alien1_stats['power'] - alien2_stats['power'],
            alien1_stats['strength_level'] - alien2_stats['strength_level'],
            alien1_stats['speed_level'] - alien2_stats['speed_level'],
            alien1_stats['intelligence'] - alien2_stats['intelligence']
        ]).reshape(1, -1)
        
        # Scale features
        features_scaled = scaler.transform(features)
        
        # Make prediction
        prediction = model.predict(features_scaled)[0]
        probability = model.predict_proba(features_scaled)[0]
        
        winner = alien1_name if prediction == 1 else alien2_name
        confidence = max(probability)
        
        return {
            'winner': winner,
            'confidence': confidence,
            'alien1_win_prob': probability[1],
            'alien2_win_prob': probability[0]
        }
    
    except Exception as e:
        return {'error': str(e)}

# Test the prediction function
test_result = predict_battle_winner('Alien X', 'Four Arms', aliens_clean, lr_model, scaler)
print(f"\nTest prediction - Alien X vs Four Arms:")
print(test_result)

test_result2 = predict_battle_winner('Atomix', 'Wildmutt', aliens_clean, lr_model, scaler)
print(f"\nTest prediction - Atomix vs Wildmutt:")
print(test_result2)

Saving the best model (Logistic Regression)...
Models saved successfully!

Test prediction - Alien X vs Four Arms:
{'winner': 'Four Arms', 'confidence': 0.5586836130185162, 'alien1_win_prob': 0.44131638698148373, 'alien2_win_prob': 0.5586836130185162}

Test prediction - Atomix vs Wildmutt:
{'winner': 'Atomix', 'confidence': 0.6284182363040343, 'alien1_win_prob': 0.6284182363040343, 'alien2_win_prob': 0.37158176369596574}


c:\Users\wajid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\wajid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [16]:
# Step 10: Create a comprehensive model evaluation report
print("="*50)
print("BEN 10 BATTLE PREDICTOR - FINAL REPORT")
print("="*50)

print(f"\nDATASET OVERVIEW:")
print(f"- Total aliens: {len(aliens_clean)}")
print(f"- Total battles: {len(battles_clean)}")
print(f"- Features used: {len(feature_columns)}")

print(f"\nMODEL PERFORMANCE COMPARISON:")
models_performance = {
    'Random Forest (All Features)': 0.4833,
    'Random Forest (Top 6 Features)': 0.5000,
    'Random Forest (Diff Features Only)': 0.5500,
    'Logistic Regression (Best)': lr_accuracy
}

for model_name, acc in models_performance.items():
    print(f"- {model_name}: {acc:.1%}")

print(f"\nFEATURE IMPORTANCE (Top 5):")
for idx, row in feature_importance.head().iterrows():
    print(f"- {row['feature']}: {row['importance']:.3f}")

print(f"\nKEY INSIGHTS:")
print("- The model achieves ~57% accuracy, which is better than random (50%)")
print("- Power difference is the most important feature")
print("- Individual alien stats are less predictive than stat differences")
print("- The battle outcomes may have more randomness/strategy than just stats")

print(f"\nMODEL FILES SAVED:")
print("- ben10_battle_predictor.pkl (Logistic Regression model)")
print("- ben10_feature_scaler.pkl (Feature scaler)")
print("- ben10_aliens_clean.pkl (Clean aliens dataset)")

print(f"\nREADY FOR DEPLOYMENT:")
print("✅ Model trained and saved")
print("✅ Prediction function created")
print("✅ Can be integrated into FastAPI backend")

BEN 10 BATTLE PREDICTOR - FINAL REPORT

DATASET OVERVIEW:
- Total aliens: 74
- Total battles: 300
- Features used: 12

MODEL PERFORMANCE COMPARISON:
- Random Forest (All Features): 48.3%
- Random Forest (Top 6 Features): 50.0%
- Random Forest (Diff Features Only): 55.0%
- Logistic Regression (Best): 56.7%

FEATURE IMPORTANCE (Top 5):
- power_diff: 0.104
- intelligence_diff: 0.099
- alien_2_power: 0.099
- speed_diff: 0.095
- alien_1_power: 0.091

KEY INSIGHTS:
- The model achieves ~57% accuracy, which is better than random (50%)
- Power difference is the most important feature
- Individual alien stats are less predictive than stat differences
- The battle outcomes may have more randomness/strategy than just stats

MODEL FILES SAVED:
- ben10_battle_predictor.pkl (Logistic Regression model)
- ben10_feature_scaler.pkl (Feature scaler)
- ben10_aliens_clean.pkl (Clean aliens dataset)

READY FOR DEPLOYMENT:
✅ Model trained and saved
✅ Prediction function created
✅ Can be integrated into FastA